In [ ]:
from tvscreener import StockScreener, ForexScreener, CryptoScreener
from tvscreener.field.stock import StockField
from tvscreener.filter import FilterOperator, ExtraFilter
from tvscreener.field import Exchange, SubMarket, SymbolType, IndexSymbol, Market

# New in v0.1.0: BondScreener, FuturesScreener, CoinScreener
from tvscreener import BondScreener, FuturesScreener, CoinScreener

%load_ext autoreload
%autoreload 2

# Stock Screener

This notebook demonstrates basic usage of the StockScreener. For new features in v0.1.0 (fluent API, field presets, field discovery, index filtering), see the **NewFeatures.ipynb** notebook.

With the first example, we gather all the stocks with the default values.

The default values match TradingView Screener: Market is 'USA', and it is sorted by market capitalization.

In [ ]:
ss = StockScreener()
df = ss.get()
df.head(10)

## Filters

Filters helps reduce the amount of results returned by TradingView. You can filter using **fields**, and/or using **extra filters**.

![title](img/stock-filters.png)

#### Fields Filtering

In [ ]:
ss = StockScreener()

In [ ]:
# Filter by exchange
ss.add_filter(StockField.EXCHANGE, FilterOperator.IN_RANGE, [Exchange.NYSE, Exchange.NASDAQ])

As a general rule, if you pass an array as value, the field operator will be `IN_RANGE`. If you pass a single value, the field operator will be `EQUAL`.
Internally, `tvscreener` will convert the field operator to the correct one.

In [ ]:
# Filter operator automatic conversion example
ss.add_filter(StockField.EXCHANGE, FilterOperator.EQUAL, Exchange.NYSE)
# Filter operator will be converted to IN_RANGE
ss.add_filter(StockField.EXCHANGE, FilterOperator.EQUAL, Exchange.NASDAQ) 

In [ ]:
# Filter by submarket
ss.add_filter(StockField.SUBMARKET, FilterOperator.IN_RANGE, [SubMarket.OTCQB, SubMarket.PINK])

# Filter by Volume
ss.add_filter(StockField.VOLUME, FilterOperator.ABOVE_OR_EQUAL, 50000)

# Filter by Basic EPS (TTM):
ss.add_filter(StockField.BASIC_EPS_TTM, FilterOperator.BELOW, 10)

# Exception for the field Symbol Type which requires extra internal controls
ss.set_symbol_types(SymbolType.COMMON_STOCK, SymbolType.ETF)

#### Extra Filters

In [ ]:
# Filter by current trading day
ss.add_filter(ExtraFilter.CURRENT_TRADING_DAY, FilterOperator.EQUAL, True)
# Filter by primary listing
ss.add_filter(ExtraFilter.PRIMARY, FilterOperator.EQUAL, True)

#### Index Filtering

Filter results to only include constituents of specific indices (S&P 500, NASDAQ 100, etc.).

In [ ]:
# Filter to S&P 500 constituents only
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)
ss.set_range(0, 500)
df = ss.get()
print(f"Found {len(df)} S&P 500 stocks")
df.head(10)

In [ ]:
# Multiple indices: NASDAQ 100 + Dow Jones
ss = StockScreener()
ss.set_index(IndexSymbol.NASDAQ_100, IndexSymbol.DOW_JONES)
df = ss.get()
print(f"Found {len(df)} stocks from NASDAQ 100 and Dow Jones")
df.head()

In [ ]:
# Search available indices
print("Semiconductor indices:")
for idx in IndexSymbol.search("semiconductor"):
    print(f"  {idx.name}: {idx.label}")

print("\nS&P 500 sector indices:")
for idx in IndexSymbol.search("sp500"):
    print(f"  {idx.name}: {idx.label}")

#### Specific Symbols

Fetch data for a specific list of symbols.

In [ ]:
# Fetch data for specific symbols (use EXCHANGE:SYMBOL format)
ss = StockScreener()
ss.symbols = {
    "query": {"types": []},
    "tickers": ["NASDAQ:AAPL", "NASDAQ:MSFT", "NASDAQ:GOOGL", "NYSE:IBM", "NYSE:JPM"]
}
df = ss.get()
print(f"Fetched data for {len(df)} symbols")
df

In [ ]:
# Fetch ALL ~3500 fields for specific symbols
ss = StockScreener()
ss.symbols = {
    "query": {"types": []},
    "tickers": ["NASDAQ:AAPL", "NASDAQ:MSFT", "NASDAQ:GOOGL"]
}
ss.select_all()  # Request all available fields
df = ss.get()
print(f"Fetched {len(df.columns)} fields for {len(df)} symbols")
df

## Search

![title](img/search.png)

In [ ]:
# Search for stocks with name matching 'AA'
ss = StockScreener()
ss.search('AA')

df = ss.get()
df.head(10)

## Sort

In [ ]:
# Sort by 'All-Time Low' descending
ss = StockScreener()
ss.sort_by(StockField.ALL_TIME_LOW, ascending=False)
ss.get()

## Technical Fields

It could be useful to see the technical column name in place of the human-readable column name.

In [ ]:
ss = StockScreener()
df = ss.get()
# Set the technical columns along with the human-readable columns
df.set_technical_columns()
df.head(10)

In [ ]:
# Or just the technical column names
df.set_technical_columns(only=True)
df.head(10)

## Get everything
To get the entire universe of stocks, set the market to 'ALL' and the range to 1 Million (TradingView manages ~102k stocks).

Use `select_all()` to fetch all ~3500 available fields.

In [ ]:
raise Warning('This will take time to render and the dataframe will be huge (~235Mb).')

In [ ]:
ss = StockScreener()
ss.set_markets(Market.ALL)
ss.set_range(0, 10**6)
ss.select_all()  # Fetch all ~3500 fields
df = ss.get()
print(f"Fetched {len(df)} stocks with {len(df.columns)} fields")
df.head(10)